# Google's Quick Draw Dataset and GANs
## Lawton Manning, John Farrell, and Liyao Zhang

*Before running this Notebook:* <br> Make sure to download the QuickDraw files corresponding to the '.npy' filetype.

More information on Google's Quick Draw dataset found in their repository here: https://github.com/googlecreativelab/quickdraw-dataset 

## Preliminary Data Prep:

In [16]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import basename, isfile, join

from matplotlib.pyplot import imshow

In [2]:
metadata = pd.read_csv(join('data', 'metadata.csv'))

In [54]:
%reload_ext autoreload
%autoreload 2
from DataGenerator import DataGenerator

datagen = DataGenerator(metadata, 32)   # locally defined Datagen class specific to Google's QuickDraw
datagen.n_classes

345

## Creating the GAN Model:
In order to model our GAN, we must define the two sub-models that comprise GANs in general. Collectively, they will learn to create sketches from Google's QuickDraw 28x28 greyscale images.
    
1. Discriminator - given an input image, determine if the image is real or fake
2. Generator - learn how to generate an image which fools the discriminator (so that it cannot distinguish real/fake reliably)
    

In [ ]:
from keras.models import Sequential, Model
from keras import Input
from keras.layers import Dense, Reshape, Flatten, Conv2D, LeakyReLU, Concatenate

def discriminator():
    dropout=0.4
    depth=64
    alpha=0.2
    
    label = Input(shape=(1,))
    x = Embedding(345,50)(label)
    x = Dense(784)(x)
    x = Reshape((28,28,1))(x)
    
    image = Input(shape=(28,28,1))
    
    concat = Concatenate()[image,x]
    x = Conv2D(1*depth,5,strides=2,padding='same')(concat)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(2*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(4*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(8*depth,5,strides=1,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Flatten()(x)
    out = Dense(1,activation='sigmoid')(x)
    
    model = Model(inputs=[image,label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
# JOHN EDITS:
# from here on I just kinda wanted to copy code and lay general groundwork for what I anticipate:

def generator():
    dropout=0.4
    depth=32 # FIXME - you mentioned changed depth I think 32?
    alpha=0.2

    # compile model
    model = Model(inputs=[image,label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [ ]:
# create submodels and compile final GAN
discriminator = discriminator()
generator = generator()
sketchGAN = Model() # yadda yadda